In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import json
import re
import sentencepiece as spm

In [4]:
from xlnet.prepro_utils import preprocess_text, encode_ids, encode_pieces

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.bert.model')

with open('sp10m.cased.bert.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [5]:
from malaya.train.model.bigbird import modeling, optimization
import numpy as np
import json
import tensorflow as tf
import itertools
from unidecode import unidecode
import re
import random

In [6]:
import json

with open('dataset.json') as fopen:
    data = json.load(fopen)

In [7]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [8]:
import re
from unidecode import unidecode

def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [9]:
from tqdm import tqdm

for i in tqdm(range(len(train_X))):
    train_X[i] = cleaning(train_X[i])
    
for i in tqdm(range(len(test_X))):
    test_X[i] = cleaning(test_X[i])

100%|██████████| 5029/5029 [00:01<00:00, 2742.14it/s]


In [10]:
MAX_SEQ_LENGTH = 1024

def get_tokenize(strings):
    input_ids, input_masks = [], []
    for text in tqdm(strings):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a) > MAX_SEQ_LENGTH - 2:
            tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)
        input_ids.append(input_id)
        input_masks.append(input_mask)
    return input_ids, input_masks

In [11]:
train_input_ids, train_input_masks = get_tokenize(train_X)
test_input_ids, test_input_masks = get_tokenize(test_X)

100%|██████████| 5029/5029 [00:20<00:00, 242.62it/s]


In [12]:
bert_config = {
    'attention_probs_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'hidden_size': 312,
    'initializer_range': 0.02,
    'intermediate_size': 1200,
    'max_position_embeddings': 4096,
    'max_encoder_length': MAX_SEQ_LENGTH,
    'num_attention_heads': 12,
    'num_hidden_layers': 4,
    'type_vocab_size': 2,
    'scope': 'bert',
    'use_bias': True,
    'rescale_embedding': False,
    'vocab_model_file': None,
    'attention_type': 'block_sparse',
    'norm_type': 'postnorm',
    'block_size': 16,
    'num_rand_blocks': 3,
    'vocab_size': 32000,
}

In [13]:
epoch = 10
batch_size = 16
warmup_proportion = 0.1
num_train_steps = int(len(train_X) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [14]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(bert_config)
        sequence_output, pooled_output = model(self.X, training = training)
        
        output_layer = sequence_output
        output_layer = tf.layers.dense(
            output_layer,
            bert_config['hidden_size'],
            activation=tf.tanh,
            kernel_initializer=create_initializer())
        self.logits_seq = tf.layers.dense(output_layer, dimension_output,
                                         kernel_initializer=create_initializer())
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)
sess.run(tf.global_variables_initializer())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use keras.layers.Dense instead.


Instructions for updating:
Use keras.layers.Dense instead.


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


In [16]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        name_r = name.replace('bert/embeddings/LayerNorm', 'bert/encoder/LayerNorm')
        if name_r not in name_to_variable:
            continue
        if 'embeddings/position_embeddings' in name_r:
            continue
        assignment_map[name] = name_to_variable[name_r]
        initialized_variable_names[name_r] = 1
        initialized_variable_names[name_r + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [17]:
tvars = tf.trainable_variables()
checkpoint = 'tiny-bert-v1/model.ckpt'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [18]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from tiny-bert-v1/model.ckpt


INFO:tensorflow:Restoring parameters from tiny-bert-v1/model.ckpt


In [19]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [20]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post', maxlen = MAX_SEQ_LENGTH)
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post', maxlen = MAX_SEQ_LENGTH)
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/3236 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.724762
time taken: 710.2374422550201
epoch: 0, training loss: 0.633651, training acc: 0.649142, valid loss: 0.553987, valid acc: 0.724762



train minibatch loop:   0%|          | 0/3236 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.724762, current acc: 0.776190
time taken: 705.942387342453
epoch: 1, training loss: 0.534513, training acc: 0.739937, valid loss: 0.483162, valid acc: 0.776190



train minibatch loop:   0%|          | 0/3236 [00:00<?, ?it/s]

...

epoch: 3, pass acc: 0.794643, current acc: 0.809921
time taken: 706.6109311580658
epoch: 3, training loss: 0.448780, training acc: 0.795021, valid loss: 0.437806, valid acc: 0.809921



train minibatch loop:   0%|          | 0/3236 [00:00<?, ?it/s]

epoch: 4, pass acc: 0.809921, current acc: 0.812698
time taken: 706.7714729309082
epoch: 4, training loss: 0.427094, training acc: 0.808676, valid loss: 0.427628, valid acc: 0.812698



test minibatch loop: 100%|██████████| 315/315 [00:23<00:00, 13.31it/s, accuracy=1, cost=0.0864]   

time taken: 706.6674127578735
epoch: 5, training loss: 0.411375, training acc: 0.816170, valid loss: 0.420944, valid acc: 0.812500

break epoch:6



In [21]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'tiny-bigbird-relevancy/model.ckpt')

'tiny-bigbird-relevancy/model.ckpt'

In [22]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate,
    training = False
)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'tiny-bigbird-relevancy/model.ckpt')

INFO:tensorflow:Restoring parameters from tiny-bigbird-relevancy/model.ckpt


INFO:tensorflow:Restoring parameters from tiny-bigbird-relevancy/model.ckpt


In [23]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'gradients' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/encoder/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/self/query/kernel',
 'bert/encoder/layer_0/attention/self/query/bias',
 'bert/encoder/layer_0/attention/self/key/kernel',
 'bert/encoder/layer_0/attention/self/key/bias',
 'bert/encoder/layer_0/attention/self/value/kernel',
 'bert/encoder/layer_0/attention/self/value/bias',
 'bert/encoder/layer_0/attention/self/Softmax',
 'bert/encoder/layer_0/attention/self/Softmax_1',
 'bert/encoder/layer_0/attention/self/Softmax_2',
 'bert/encoder/layer_0/attention/self/Softmax_3',
 'bert/encoder/layer_0/attention/self/Softmax_4',
 'bert/encoder/layer_0/attention/output/dense/kernel',
 'bert/encoder/layer_0/attention/output/dense/bias',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/intermediate/dense/kernel',
 'bert/encoder/layer_0/intermediate/dense/

In [24]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post', maxlen = MAX_SEQ_LENGTH)
    batch_y = test_Y[i: index]
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 315/315 [00:22<00:00, 13.83it/s]


In [25]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['not relevant', 'relevant'],
        digits = 5
    )
)

              precision    recall  f1-score   support

not relevant    0.91691   0.64322   0.75605      1990
    relevant    0.80457   0.96183   0.87620      3039

    accuracy                        0.83575      5029
   macro avg    0.86074   0.80252   0.81613      5029
weighted avg    0.84902   0.83575   0.82866      5029



In [26]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [27]:
freeze_graph('tiny-bigbird-relevancy', strings)

INFO:tensorflow:Restoring parameters from tiny-bigbird-relevancy/model.ckpt


INFO:tensorflow:Restoring parameters from tiny-bigbird-relevancy/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

INFO:tensorflow:Froze 75 variables.


INFO:tensorflow:Froze 75 variables.


INFO:tensorflow:Converted 75 variables to const ops.


INFO:tensorflow:Converted 75 variables to const ops.


4948 ops in the final graph.


In [28]:
from tensorflow.tools.graph_transforms import TransformGraph

In [29]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [30]:
pb = 'tiny-bigbird-relevancy/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())
        
inputs = ['Placeholder', 'Placeholder_1']
outputs = ['dense/BiasAdd']
transformed_graph_def = TransformGraph(input_graph_def, 
                                       inputs,
                                       ['logits', 'logits_seq'] + outputs, transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


Instructions for updating:
Use tf.gfile.GFile.


In [32]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


In [33]:
g = load_graph('tiny-bigbird-relevancy/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')

In [34]:
x, logits

(<tf.Tensor 'import/Placeholder:0' shape=(?, ?) dtype=int32>,
 <tf.Tensor 'import/logits:0' shape=(?, 2) dtype=float32>)

In [35]:
test_sess = tf.InteractiveSession(graph = g)

In [36]:
%%time
test_sess.run(logits, feed_dict = {x: [[1] * MAX_SEQ_LENGTH]})

CPU times: user 777 ms, sys: 78.2 ms, total: 856 ms
Wall time: 842 ms


array([[ 0.6969471, -0.9225403]], dtype=float32)

In [37]:
g = load_graph('tiny-bigbird-relevancy/frozen_model.pb.quantized')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

In [38]:
%%time
test_sess.run(logits, feed_dict = {x: [[1] * MAX_SEQ_LENGTH]})

CPU times: user 957 ms, sys: 171 ms, total: 1.13 s
Wall time: 1.01 s


array([[ 0.20388116, -0.41363555]], dtype=float32)